In [2]:
import pandas as pd
import numpy as np
pd.set_option('precision', 5)

In [3]:
#read in the data
master_app = pd.read_csv('/home/emilyboeke/merged_full_asylum_master_app.csv')
#load the idncases in the training set, and select the rows of master_app that correspond to those cases.
path = '/home/emilyboeke/'

train_cases = pd.read_csv(path + 'train_cases_any_asylum.csv',header=None).values
train_cases = train_cases.reshape(train_cases.shape[0])
master_app = master_app[master_app.idncase.isin(train_cases)]


master_app['osc_date'] = pd.to_datetime(master_app['osc_date'],infer_datetime_format = True)


In [4]:
master_app.head(5)

,idncase,idnproceeding,idnProceedingAppln,numAppsPerProc,dec,nat,case_type,c_asy_type,base_city_code,hearing_loc_code,osc_date,comp_date,attorney_flag,tracid,numProcPerCase
0,2046931.0,3200059.0,803710,4.0,DENY,CH,RMV,def,CHI,CHD,2003-09-09,2004-02-20,1.0,32.0,1
1,2046932.0,3371117.0,2819106,3.0,DENY,CH,RMV,def,NYC,NYC,2003-09-09,2005-05-13,1.0,148.0,1
2,2046933.0,3200063.0,3390354,3.0,DENY,CH,RMV,def,CLE,CEL,2003-09-16,2007-08-15,1.0,3.0,1
3,2046934.0,3200066.0,803711,2.0,DENY,CM,RMV,def,CHI,CHD,2003-09-24,2004-01-06,1.0,31.0,1
4,2046935.0,3200067.0,1681269,2.0,DENY,CH,AOC,def,CHI,CHD,2003-09-08,2004-01-30,1.0,32.0,1


In [4]:
#nat time features
#for each nat, make a feature that is the average grant rate for all past decisions
#sort by nat, then date. 
#for each nat date combo,count grant and deny.
#do a cumulative sum


#group by nat, then by date, and record the number deny and number grant for that nat/date combo.
master_app = master_app.sort_values(['nat','comp_date'])
dec_counts = master_app.groupby(['nat','comp_date'],as_index=False).dec.agg({'g': lambda x: len(x[x =='GRANT']),
                                                                             'd': lambda x: len(x[x =='DENY'])})


In [5]:
dec_counts['comp_date'] = pd.to_datetime(dec_counts['comp_date'],infer_datetime_format = True)


In [75]:
#have  to do this separately for test data


nationalities = dec_counts.nat.unique()

dec_counts['nat_num_prev']=np.zeros(len(dec_counts))
dec_counts['nat_gr_past']=np.zeros(len(dec_counts))
dec_counts['nat_num_lastyear']=np.zeros(len(dec_counts))
dec_counts['nat_gr_lastyear']=np.zeros(len(dec_counts))

#cycle through nationalities and calculate total past grant rate, grant rate in the last year, and num decisions
for n in nationalities:
    g_cumulative = np.cumsum(dec_counts.loc[dec_counts['nat']==n,'g'])
    g_current = dec_counts.loc[dec_counts['nat']==n,'g']
    g_past =  g_cumulative-g_current #num past grants
    d_cumulative = np.cumsum(dec_counts.loc[dec_counts['nat']==n,'d'])
    d_current = dec_counts.loc[dec_counts['nat']==n,'d']
    d_past =  d_cumulative-d_current# num past denials
    num_prev = g_past + d_past# number of past decisions
    gr_past = g_past / num_prev #past grant rate
    #store this info in the df
    dec_counts.loc[dec_counts['nat']==n,'nat_num_prev'] = num_prev
    dec_counts.loc[dec_counts['nat']==n,'nat_gr_past'] = gr_past
    
    #cycle through dates and  calculate grant rate in the last year, and num decisions
    dates = dec_counts.loc[dec_counts['nat']==n,'comp_date']
    dates_1back = dates - pd.DateOffset(years=1) #calculate dates - 1 year
    dec_counts_nat = dec_counts.loc[dec_counts['nat']==n]
    num_lastyear = np.zeros(len(dates))
    gr_lastyear = np.zeros(len(dates))
    for i in range(len(dates)):
        #count num grants after 1 year ago but before today
        g_lastyear = len(dec_counts_nat.loc[[(dec_counts_nat['comp_date'] > dates_1back[i]) 
                                         & (dec_counts_nat['comp_date'] < dates[i])],'g'])
        #count num denies after 1 year ago but before today
        d_lastyear = len(dec_counts_nat.loc[[(dec_counts_nat['comp_date'] > dates_1back[i]) 
                                         & (dec_counts_nat['comp_date'] < dates[i])],'d'])
        num_lastyear[i] = g_lastyear + d_lastyear # num decisions in the last year
        
        gr_lastyear[i] = g_lastyear / num_lastyear #grant rate in the last year
    
    #store this info in the df
    dec_counts.loc[dec_counts['nat']==n,'nat_num_lastyear'] = num_lastyear
    dec_counts.loc[dec_counts['nat']==n,'nat_gr_lastyear'] = gr_lastyear
    
    


ValueError: too many values to unpack (expected 1)

In [1]:
#bla = dec_counts.loc[dec_counts['nat']=='AC','comp_date'] 
#bla2 = bla - pd.DateOffset(years=1)
[(dec_counts.comp_date>dec_counts.comp_date[20]) & (dec_counts.comp_date<dec_counts.comp_date[0])]
d_lastyear = len(dec_counts_nat.loc[[(dec_counts_nat['comp_date'] > dates_1back[i]) 
                                         & (dec_counts_nat['comp_date'] < dates[i])],'d'])

NameError: name 'dec_counts' is not defined

In [57]:
dec_counts.loc[dec_counts['nat']=='AC','comp_date']

0   1996-09-04
1   2000-01-31
2   2008-06-12
3   2010-09-16
4   2010-10-04
5   2011-03-07
6   2012-01-12
7   2012-06-21
Name: comp_date, dtype: datetime64[ns]

In [34]:
#now match these num_prev and gr_past values back to the master_app data frame
temp = dec_counts.drop(['d','g'],axis=1)
master_app = pd.merge(master_app,temp,how='left',on=['nat','comp_date'])

In [40]:
dec_counts.head(20)

,nat,comp_date,g,d,num_prev,gr_past
0,AC,1996-09-04,0,1,0.0,NaN
1,AC,2000-01-31,0,1,1.0,0.00000
2,AC,2008-06-12,0,1,2.0,0.00000
3,AC,2010-09-16,0,1,3.0,0.00000
4,AC,2010-10-04,1,0,4.0,0.00000
5,AC,2011-03-07,0,1,5.0,0.20000
6,AC,2012-01-12,0,1,6.0,0.16667
7,AC,2012-06-21,0,1,7.0,0.14286
8,AF,1985-02-01,1,0,0.0,NaN
9,AF,1985-02-22,1,0,1.0,1.00000


,idncase,idnproceeding,idnProceedingAppln,numAppsPerProc,dec,nat,case_type,c_asy_type,base_city_code,hearing_loc_code,osc_date,comp_date,attorney_flag,tracid,numProcPerCase,num_prev,gr_past
0,3365575.0,1663037.0,386078,3.0,DENY,AC,DEP,aff,NYC,NYC,1996-04-25,1996-09-04,1.0,276.0,1,0.0,NaN
1,2804255.0,1093167.0,189326,3.0,DENY,AC,RMV,def,BUF,BTV,1999-07-08,2000-01-31,0.0,12.0,1,1.0,0.00000
2,5958103.0,4493631.0,3541380,3.0,DENY,AC,RMV,def,KRO,KRO,2006-10-06,2008-06-12,0.0,77.0,1,2.0,0.00000
3,6493810.0,5169760.0,3993862,3.0,DENY,AC,RMV,def,KRO,GCP,2010-04-21,2010-09-16,1.0,77.0,1,3.0,0.00000
4,6111082.0,4686271.0,3647884,1.0,GRANT,AC,RMV,aff,PHO,PHO,2008-11-20,2010-10-04,1.0,176.0,1,4.0,0.00000
5,6300832.0,5220172.0,3998537,3.0,DENY,AC,RMV,def,KRO,GCP,2009-07-27,2011-03-07,1.0,77.0,1,5.0,0.20000
6,6794748.0,5651337.0,4305365,3.0,DENY,AC,RMV,def,KRO,KRO,2011-05-12,2012-01-12,0.0,124.0,1,6.0,0.16667
7,5932843.0,5473368.0,4412556,4.0,DENY,AC,RMV,def,YOR,YOR,2008-02-20,2012-06-21,1.0,280.0,1,7.0,0.14286
8,2246087.0,366669.0,75759,2.0,GRANT,AF,EXC,def,SFR,SFR,1984-01-04,1985-02-01,0.0,202.0,1,0.0,NaN
9,2246083.0,367396.0,76057,2.0,GRANT,AF,EXC,def,SFR,SFR,1984-05-02,1985-02-22,1.0,312.0,1,1.0,1.00000


,idncase,idnproceeding,idnProceedingAppln,numAppsPerProc,dec,nat,case_type,c_asy_type,base_city_code,hearing_loc_code,osc_date,comp_date,attorney_flag,tracid,numProcPerCase,nat_num_prev,nat_gr_past
0,3365575.0,1663037.0,386078,3.0,DENY,AC,DEP,aff,NYC,NYC,1996-04-25,1996-09-04,1.0,276.0,1,0.0,NaN
1,2804255.0,1093167.0,189326,3.0,DENY,AC,RMV,def,BUF,BTV,1999-07-08,2000-01-31,0.0,12.0,1,1.0,0.0
2,5958103.0,4493631.0,3541380,3.0,DENY,AC,RMV,def,KRO,KRO,2006-10-06,2008-06-12,0.0,77.0,1,2.0,0.0
3,6493810.0,5169760.0,3993862,3.0,DENY,AC,RMV,def,KRO,GCP,2010-04-21,2010-09-16,1.0,77.0,1,3.0,0.0
4,6111082.0,4686271.0,3647884,1.0,GRANT,AC,RMV,aff,PHO,PHO,2008-11-20,2010-10-04,1.0,176.0,1,4.0,0.0
